<a href="https://colab.research.google.com/github/ErikFantomex/Chatterbot-TIAI/blob/main/Chatbot_TAIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento

In [ ]:
!pip install chatterbot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 269 kB 48.3 MB/s 
     |████████████████████████████████| 265 kB 66.0 MB/s 
     |████████████████████████████████| 225 kB 68.0 MB/s 
     |████████████████████████████████| 5.7 MB 49.0 MB/s 
     |████████████████████████████████| 30.7 MB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 526 kB 31.1 MB/s 
  Using cached blis-0.2.4-cp38-cp38-linux_x86_64.whl
  Using cached srsly-1.0.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (211 kB)
  Using cached thinc-7.0.8-cp38-cp38-linux_x86_64.whl
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Using cached preshed-2.0.1-cp38-cp38-linux_x86_64.whl
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp38-cp38-

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
import re
import requests
from bs4 import BeautifulSoup

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

In [11]:

#iniciamos
url = 'https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html'
page = requests.get(url)
page = page.text
soup = BeautifulSoup(page, features="lxml")
for script in soup(["script", "style"]):
    script.decompose()
raw_text  = soup.get_text()

cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
cleaned = re.sub(r" ", " ", cleaned)
cleaned = re.sub(r"  ", " ", cleaned)
cleaned = re.sub("\n", " ", cleaned)
cleaned = cleaned.strip()

tokens = nltk.sent_tokenize(cleaned)
for token in tokens:
  print('-> ', token, '\n')
     


->  Pathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough Performance – Google AI BlogSkip to main contentPathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough PerformancePathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough Performance Philosophy Research Areas Publications People Tools & Downloads Outreach Careers Blog ArchiveLabelsAllABCDEFGHIJKLMNOPQRSTUVWXYZBlog          ›         Pathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough PerformanceMonday, April 04, 2022Posted by Sharan Narang and Aakanksha Chowdhery, Software Engineers, Google ResearchIn recent years, large neural networks trained for language understanding and generation have achieved impressive results across a wide range of tasks. 

->  GPT-3 first showed that large language models (LLMs) can be used for few-shot learning and can achieve impressive results without large-scale task-specific data colle

In [12]:




url = 'https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html'
page = requests.get(url)
page = page.text
soup = BeautifulSoup(page, features="lxml")
for script in soup(["script", "style"]):
    script.decompose()
raw_text  = soup.get_text()

cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
cleaned = re.sub(r" ", " ", cleaned)
cleaned = re.sub(r"  ", " ", cleaned)
cleaned = re.sub("\n", " ", cleaned)
cleaned = re.sub("'", " ", cleaned)
cleaned = re.sub("\"", " ", cleaned)
cleaned = cleaned.strip()
     


# Vectorizacion

In [13]:
sent_tokens = nltk.sent_tokenize(cleaned)
word_tokens = nltk.word_tokenize(cleaned)

In [14]:
import math

total_ocurrences = {}
occured_in_n_docs = {}

for sentence in sent_tokens:
  for word in word_tokens:
 
    c = sentence.count(word)
 
    try:
      total_ocurrences[word] += c
    except KeyError:
      total_ocurrences[word] = c 
 
    if c > 0:
      try:
        occured_in_n_docs[word] += 1
      except KeyError:
        occured_in_n_docs[word] = 1
 

tfidf = []
i = 0
for sentence in sent_tokens:
  w = {"sentence_n": i}
  for word in word_tokens:
    # w = tf * log(N/df)
    tf = sentence.count(word)
    N = total_ocurrences[word]
    df = occured_in_n_docs[word]
    if tf == 0:
      w[word] = 0
    else:
      w[word] = tf * math.log(N/df)
  tfidf.append(w)
  i += 1


In [ ]:
pd.DataFrame(tfidf)
     

# Similaridad de coseno

In [16]:
url = 'https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html'
page = requests.get(url)
page = page.text
soup = BeautifulSoup(page, features="lxml")
for script in soup(["script", "style"]):
    script.decompose()
raw_text  = soup.get_text()

cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
cleaned = re.sub(r" ", " ", cleaned)
cleaned = re.sub(r"  ", " ", cleaned)
cleaned = re.sub("\n", " ", cleaned)
cleaned = re.sub("'", " ", cleaned)
cleaned = re.sub("\"", " ", cleaned)
cleaned = cleaned.strip()

sent_tokens = nltk.sent_tokenize(cleaned)

In [17]:

similaridades = []
for sentece in sent_tokens:
  row = []
  A = {w for w in nltk.word_tokenize(sentece)}
  for another_sentence in sent_tokens:
    l1 =[]
    l2 =[]
    B = {w for w in nltk.word_tokenize(another_sentence)}
    v = A.union(B) 
    for w in v:
      l1.append(1 if w in A else 0)
      l2.append(1 if w in B else 0)
    c = 0
    for i in range(len(v)):
        c+= l1[i]*l2[i]
    row.append(c / float((sum(l1) * sum(l2)) ** 0.5))
  similaridades.append(row)

In [20]:
pd.DataFrame(similaridades)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1.000000,0.197814,0.162791,0.222911,0.107676,0.269880,0.250259,0.209381,0.096309,0.166839,...,0.122868,0.159256,0.180579,0.225920,0.238352,0.269191,0.072798,0.191533,0.109897,0.129732
1,0.197814,1.000000,0.185164,0.197203,0.081650,0.146176,0.172516,0.173205,0.146059,0.161015,...,0.149071,0.207020,0.121716,0.190347,0.154919,0.122474,0.047316,0.064550,0.100000,0.065583
2,0.162791,0.185164,1.000000,0.182574,0.037796,0.081200,0.287494,0.133631,0.202837,0.234255,...,0.138013,0.095831,0.197203,0.211472,0.191237,0.188982,0.043806,0.179284,0.154303,0.060718
3,0.222911,0.197203,0.182574,1.000000,0.172516,0.247083,0.291606,0.292770,0.216025,0.194404,...,0.188982,0.233285,0.257172,0.353919,0.327327,0.310530,0.066649,0.245495,0.225374,0.166282
4,0.107676,0.081650,0.037796,0.172516,1.000000,0.179029,0.042258,0.247487,0.089443,0.140859,...,0.136931,0.042258,0.186339,0.093250,0.221359,0.100000,0.077267,0.158114,0.204124,0.120483
5,0.269880,0.146176,0.081200,0.247083,0.179029,1.000000,0.121046,0.303822,0.256205,0.302614,...,0.163430,0.121046,0.507072,0.233723,0.317038,0.179029,0.082998,0.198148,0.116941,0.172559
6,0.250259,0.172516,0.287494,0.291606,0.042258,0.121046,1.000000,0.179284,0.151186,0.214286,...,0.154303,0.250000,0.188982,0.394055,0.213809,0.295804,0.081628,0.267261,0.207020,0.067884
7,0.209381,0.173205,0.133631,0.292770,0.247487,0.303822,0.179284,1.000000,0.221359,0.239046,...,0.193649,0.149404,0.368932,0.329690,0.290689,0.212132,0.068295,0.167705,0.144338,0.141990
8,0.096309,0.146059,0.202837,0.216025,0.089443,0.256205,0.151186,0.221359,1.000000,0.277174,...,0.244949,0.151186,0.366667,0.250217,0.197990,0.313050,0.069109,0.176777,0.146059,0.107763
9,0.166839,0.161015,0.234255,0.194404,0.140859,0.302614,0.214286,0.239046,0.277174,1.000000,...,0.154303,0.142857,0.230978,0.183892,0.231626,0.197203,0.054419,0.155902,0.184017,0.090513


# Chatbot

In [ ]:
bot = ChatBot(
    'ReporterBot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3'
    )
    
chat = ListTrainer(bot)
chat.train(all_senteces)

In [ ]:
while True:
  response = bot.get_response(input("Pergunta: "))
  print("[Confiança na resposta: {:}%]\nResposta:".format(100*float(response.confidence)), response)